<a href="https://colab.research.google.com/github/Vishy-A/DeepLearning/blob/main/HW3/4106HW3P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [ ]:
seq = "Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text. At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model. One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character prediction tasks. Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction plays a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future of text-based technology."

In [ ]:
chars  = sorted(list(set(seq)))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}
chars = sorted(list(set(seq)))

In [ ]:
def definition(maxlen):
  x = []
  y = []
  for i in range(len(seq) - maxlen):
      sequence = seq[i:i + maxlen]
      label = seq[i + maxlen]
      x.append([char_to_int[char] for char in sequence])
      y.append(char_to_int[label])

  x = np.array(x)
  y = np.array(y)

  return x, y

In [ ]:
x10, y10 = definition(10)
x20, y20 = definition(20)
x30, y30 = definition(30)

In [ ]:
x10train, x10test, y10train, y10test = train_test_split(x10, y10, test_size=0.2, random_state=42)
x20train, x20test, y20train, y20test = train_test_split(x20, y20, test_size=0.2, random_state=42)
x30train, x30test, y30train, y30test = train_test_split(x30, y30, test_size=0.2, random_state=42)


In [ ]:
x10train = torch.tensor(x10train, dtype=torch.long)
x10test = torch.tensor(x10test, dtype=torch.long)
y10train = torch.tensor(y10train, dtype=torch.long)
y10test = torch.tensor(y10test, dtype=torch.long)

x20train = torch.tensor(x20train, dtype=torch.long)
x20test = torch.tensor(x20test, dtype=torch.long)
y20train = torch.tensor(y20train, dtype=torch.long)
y20test = torch.tensor(y20test, dtype=torch.long)

x30train = torch.tensor(x30train, dtype=torch.long)
x30test = torch.tensor(x30test, dtype=torch.long)
y30train = torch.tensor(y30train, dtype=torch.long)
y30test = torch.tensor(y30test, dtype=torch.long)


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        embedded = self.embedding(input)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        return output

In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.lstm = nn.LSTM(hidden_size, hidden_size)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, input):
    embedded = self.embedding(input)
    output, _ = self.lstm(embedded)
    output = self.fc(output[:, -1, :])
    return output

In [ ]:
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(GRU, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, input):
    embedded = self.embedding(input)
    output, _ = self.gru(embedded)
    output = self.fc(output[:, -1, :])
    return output

In [ ]:
hidden_size = 128
lr = .001
epochs = 100

criterion = nn.CrossEntropyLoss()


In [ ]:
modelRNN = RNN(len(chars), hidden_size, len(chars))
modelLSTM = LSTM(len(chars), hidden_size, len(chars))
modelGRU = GRU(len(chars), hidden_size, len(chars))

optimizerRNN = optim.Adam(modelRNN.parameters(), lr=lr)
optimizerLSTM = optim.Adam(modelLSTM.parameters(), lr=lr)
optimizerGRU = optim.Adam(modelGRU.parameters(), lr=lr)

In [ ]:
for epoch in range(epochs):
  modelRNN.train()
  optimizerRNN.zero_grad()
  outputRNN = modelRNN(x30train)
  lossRNN = criterion(outputRNN, y30train)
  lossRNN.backward()
  optimizerRNN.step()

  modelRNN.eval()
  with torch.no_grad():
    outputRNN = modelRNN(x30test)
    lossRNN = criterion(outputRNN, y30test)
    _, predictedRNN = torch.max(outputRNN.data, 1)
    accuracyRNN = (predictedRNN == y30test).sum().item() / len(y30test)

    if (epoch + 1) % 10 == 0:
      print(f'Epoch [{epoch + 1}/{epochs}], Loss: {lossRNN.item():.4f}, Accuracy: {accuracyRNN:.4f}')


Epoch [10/100], Loss: 3.0167, Accuracy: 0.1928
Epoch [20/100], Loss: 2.7299, Accuracy: 0.2309
Epoch [30/100], Loss: 2.6332, Accuracy: 0.2309
Epoch [40/100], Loss: 2.5732, Accuracy: 0.2436
Epoch [50/100], Loss: 2.5514, Accuracy: 0.2436
Epoch [60/100], Loss: 2.5405, Accuracy: 0.2331
Epoch [70/100], Loss: 2.5407, Accuracy: 0.2288
Epoch [80/100], Loss: 2.5469, Accuracy: 0.2331
Epoch [90/100], Loss: 2.5586, Accuracy: 0.2182
Epoch [100/100], Loss: 2.5766, Accuracy: 0.2140


In [ ]:
for epoch in range(epochs):
  modelLSTM.train()
  optimizerLSTM.zero_grad()
  outputLSTM = modelLSTM(x30train)
  lossLSTM = criterion(outputLSTM, y30train)
  lossLSTM.backward()
  optimizerLSTM.step()

  modelLSTM.eval()
  with torch.no_grad():
    outputLSTM = modelLSTM(x30test)
    lossLSTM = criterion(outputLSTM, y30test)
    _, predictedLSTM = torch.max(outputLSTM.data, 1)
    accuracyLSTM = (predictedLSTM == y30test).sum().item() / len(y30test)

    if (epoch + 1) % 10 == 0:
      print(f'Epoch [{epoch + 1}/{epochs}], Loss: {lossLSTM.item():.4f}, Accuracy: {accuracyLSTM:.4f}')


Epoch [10/100], Loss: 3.3896, Accuracy: 0.2013
Epoch [20/100], Loss: 2.9030, Accuracy: 0.2097
Epoch [30/100], Loss: 2.7631, Accuracy: 0.2479
Epoch [40/100], Loss: 2.6742, Accuracy: 0.2521
Epoch [50/100], Loss: 2.6005, Accuracy: 0.2542
Epoch [60/100], Loss: 2.5591, Accuracy: 0.2627
Epoch [70/100], Loss: 2.5374, Accuracy: 0.2606
Epoch [80/100], Loss: 2.5257, Accuracy: 0.2458
Epoch [90/100], Loss: 2.5279, Accuracy: 0.2331
Epoch [100/100], Loss: 2.5373, Accuracy: 0.2309


In [ ]:
for epoch in range(epochs):
  modelGRU.train()
  optimizerGRU.zero_grad()
  outputGRU = modelGRU(x30train)
  lossGRU = criterion(outputGRU, y30train)
  lossGRU.backward()
  optimizerGRU.step()

  modelGRU.eval()
  with torch.no_grad():
      outputGRU = modelGRU(x30test)
      lossGRU = criterion(outputGRU, y30test)
      _, predictedGRU = torch.max(outputGRU.data, 1)
      accuracyGRU = (predictedGRU == y30test).sum().item() / len(y30test)

      if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {lossGRU.item():.4f}, Accuracy: {accuracyGRU:.4f}')

Epoch [10/100], Loss: 3.1313, Accuracy: 0.1907
Epoch [20/100], Loss: 2.7971, Accuracy: 0.2436
Epoch [30/100], Loss: 2.6734, Accuracy: 0.2331
Epoch [40/100], Loss: 2.5834, Accuracy: 0.2373
Epoch [50/100], Loss: 2.5368, Accuracy: 0.2352
Epoch [60/100], Loss: 2.5186, Accuracy: 0.2394
Epoch [70/100], Loss: 2.5055, Accuracy: 0.2394
Epoch [80/100], Loss: 2.5039, Accuracy: 0.2331
Epoch [90/100], Loss: 2.5088, Accuracy: 0.2331
Epoch [100/100], Loss: 2.5240, Accuracy: 0.2288


In [ ]:
def predict_next_char(model, char_to_int, int_to_char, initialstr):
  model.eval()
  with torch.no_grad():
    initial_input = torch.tensor([char_to_int[char] for char in initialstr], dtype=torch.long).unsqueeze(0)
    output = model(initial_input)
    predicted = torch.argmax(output, dim=1).item()
    return int_to_char[predicted]

In [ ]:
test_str = "This is a simple example to demonstrate how to predict the next chara"
predicted_char = predict_next_char(modelGRU, char_to_int, int_to_char, test_str)
print(f"Predicted character: {predicted_char}")

Predicted character: c
